In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import pickle
import pymysql

In [42]:
db_name = "gis_system"
db_host = "localhost"
db_username = "root"
db_password = ""

try:
    conn = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
    print ("error")

# Query untuk mendapatkan data bulan sebelumnya
query = """
SELECT 
    DATE(waktu) AS tanggal,
    AVG(suhu) AS avg_temperature,
    MIN(suhu) AS min_temperature,
    MAX(suhu) AS max_temperature,
    AVG(kelembapan) AS avg_humidity,
    MIN(kelembapan) AS min_humidity,
    MAX(kelembapan) AS max_humidity
FROM forecast_weekly
WHERE DATE_FORMAT(waktu, '%Y-%m') = DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 1 MONTH), '%Y-%m')
GROUP BY DATE(waktu)
"""

df = pd.read_sql(query, conn)
df = df.dropna()

connection successful


C:\Users\ferdy\AppData\Local\Temp\ipykernel_25492\1781056883.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [43]:
monthly_weather = df.agg({
    'avg_temperature': ['mean', 'min', 'max'],
    'avg_humidity': ['mean', 'min', 'max']
}).reset_index()

print(monthly_weather.columns)

Index(['index', 'avg_temperature', 'avg_humidity'], dtype='object')


In [44]:
df_crop = pd.read_csv('../../dataset/crop_recommendation.csv')

In [45]:
crop_mapping = {
    'rice': 'padi', 'maize': 'jagung', 'chickpea': 'kacang arab', 'kidneybeans': 'kacang merah',
    'pigeonpeas': 'kacang gude', 'mothbeans': 'kacang ngengat', 'mungbean': 'kacang hijau',
    'blackgram': 'kacang hitam', 'lentil': 'kacang lentil', 'pomegranate': 'delima',
    'banana': 'pisang', 'mango': 'mangga', 'grapes': 'anggur', 'watermelon': 'semangka',
    'muskmelon': 'blewah', 'apple': 'apel', 'orange': 'jeruk', 'papaya': 'pepaya',
    'coconut': 'kelapa', 'cotton': 'kapas', 'jute': 'rami', 'coffee': 'kopi'
}
df_crop['label'] = df_crop['label'].replace(crop_mapping)

In [46]:
mean_temperature = monthly_weather['avg_temperature'].mean()
min_temperature = monthly_weather['avg_temperature'].min()
max_temperature = monthly_weather['avg_temperature'].max()

mean_humidity = monthly_weather['avg_humidity'].mean()
min_humidity = monthly_weather['avg_humidity'].min()
max_humidity = monthly_weather['avg_humidity'].max()

In [47]:
df_crop['mean_temperature'] = mean_temperature
df_crop['min_temperature'] = min_temperature
df_crop['max_temperature'] = max_temperature

df_crop['mean_humidity'] = mean_humidity
df_crop['min_humidity'] = min_humidity
df_crop['max_humidity'] = max_humidity

In [48]:
features = df_crop[['temperature', 'humidity', 'rainfall', 
                    'mean_temperature', 'mean_humidity']]
target = df_crop['label']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [50]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [51]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

        anggur       1.00      1.00      1.00        14
          apel       0.88      1.00      0.94        23
        blewah       1.00      1.00      1.00        17
        delima       0.95      0.83      0.88        23
        jagung       0.88      0.71      0.79        21
         jeruk       0.75      0.86      0.80        14
   kacang arab       1.00      0.92      0.96        26
   kacang gude       1.00      0.74      0.85        23
  kacang hijau       1.00      0.79      0.88        19
  kacang hitam       0.77      1.00      0.87        20
 kacang lentil       0.85      1.00      0.92        11
  kacang merah       0.91      1.00      0.95        20
kacang ngengat       1.00      0.79      0.88        24
         kapas       0.81      1.00      0.89        17
        kelapa       0.81      0.96      0.88        27
          kopi       0.89      0.94      0.91        17
        mangga       0.90      1.00      0.95  

In [52]:
model_filename = '../../model/ModelClassifier.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)